In [1]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer

In [2]:
df = pd.read_csv('missionData_llm.csv')
print(df.head())

                                              prompt  \
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   

                                          completion  label  
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   True  


In [3]:
completion_lst = df['completion'].tolist()

In [5]:
new_df = pd.DataFrame(columns=['chosen_response', 'rejected_response'])
chosen_lst = []
rejected_lst = [] 

for i in range(50): 
    chosen_lst.append(completion_lst[i])
    rejected_lst.append(completion_lst[i+50])

new_df['chosen_response'] = chosen_lst
new_df['rejected_response'] = rejected_lst

In [6]:
print(new_df.head())

                                     chosen_response  \
0  <bos><start_of_turn>user\nBLUELAND is a make-b...   
1  <bos><start_of_turn>user\nBLUELAND is a make-b...   
2  <bos><start_of_turn>user\nBLUELAND is a make-b...   
3  <bos><start_of_turn>user\nBLUELAND is a make-b...   
4  <bos><start_of_turn>user\nBLUELAND is a make-b...   

                                   rejected_response  
0  <bos><start_of_turn>user\nBLUELAND is a make-b...  
1  <bos><start_of_turn>user\nBLUELAND is a make-b...  
2  <bos><start_of_turn>user\nBLUELAND is a make-b...  
3  <bos><start_of_turn>user\nBLUELAND is a make-b...  
4  <bos><start_of_turn>user\nBLUELAND is a make-b...  


In [7]:
#Select a base model whch we need to train for reward modeling.
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def formatting_func(row):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = row["chosen_response"]
    prompt_plus_rejected_response = row["rejected_response"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }
    
formatted_dataset = new_df.apply(formatting_func, axis=1)
formatted_dataset = pd.DataFrame(list(formatted_dataset))

In [17]:
print(formatted_df.iloc[0, :])

input_ids_chosen           [tensor(0), tensor(41552), tensor(31957), tens...
attention_mask_chosen      [tensor(1), tensor(1), tensor(1), tensor(1), t...
input_ids_rejected         [tensor(0), tensor(41552), tensor(31957), tens...
attention_mask_rejected    [tensor(1), tensor(1), tensor(1), tensor(1), t...
Name: 0, dtype: object


In [19]:
from datasets import Dataset
formatted_dict = {
    'input_ids_chosen': formatted_df['input_ids_chosen'].tolist(), 
    'attention_mask_chosen': formatted_df['attention_mask_chosen'].tolist(), 
    'input_ids_rejected': formatted_df['input_ids_rejected'].tolist(), 
    'attention_mask_rejected': formatted_df['attention_mask_rejected'].tolist()
}
formatted_dataset = Dataset.from_dict(formatted_dict)

In [20]:
print(formatted_dataset)

Dataset({
    features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
    num_rows: 50
})


In [21]:
formatted_dataset = formatted_dataset.train_test_split()
# Configuring the training arguments
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,
)

# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

  ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Development (UC)\.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
1,0.680200,0.690945,1.000000
2,0.680500,0.687322,1.000000
3,0.673500,0.682573,1.000000
4,0.696300,0.673186,1.000000
5,0.693500,0.654999,1.000000
6,0.638300,0.621660,1.000000
7,0.646700,0.557899,1.000000
8,0.566200,0.450677,1.000000
9,0.535200,0.327528,1.000000
10,0.382700,0.205484,1.000000


TrainOutput(global_step=30, training_loss=0.23976782320532947, metrics={'train_runtime': 592.9194, 'train_samples_per_second': 0.624, 'train_steps_per_second': 0.051, 'total_flos': 0.0, 'train_loss': 0.23976782320532947, 'epoch': 10.0})

In [22]:
trainer.save_model("./rewardmodel1")

Source Code: https://medium.com/towards-generative-ai/reward-model-training-2209d1befb5f

**MODEL INFERENCE TEST 1**

In [27]:
from trl import KTOConfig, KTOTrainer, ModelConfig, get_peft_config, setup_chat_format
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-7b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
prompt_text = """
BLUELAND is a make-believe country. REDLAND is another make-believe country, an enemy of BLUELAND. 

Write me a training scenario for an air strike mission by BLUELAND to destroy a specified target. This scenario will be given to BLUELAND, who will devise a strategy to execute the plan. The scenario should be non-descriptive, concise and factual. The information should be sufficient to help BLUELAND to decide their loadout and strategy. The scenario should include the following information about the target and REDLAND: 
1.	Commander’s Intent: Captures the broad mission objectives (eg: 100% TOT, 50% BLUELAND losses, 100% REDLAND destruction, destroy bunkers, etc)
2.	Force Composition: REDLAND’s units and objects participating in the scenario, Order of Battle, i.e the identification, strength, command structure, disposition for the personnel, units, equipment of any military force, command & control hierarchies, communication networks and communication channels. This should be specific, i.e exact model of aircraft or missiles (eg: F-15 fighter jets, SA-7 surface-to-air missile launchers, ZSU-23-4 Shilka self-propelled anti-aircraft gun systems, etc). 
3.	Geography: Distance to target and distance to REDLAND threats. Specification of environmental conditions which are of interest (eg: day/night/dawn, buildings, vegetation, ditches in surroundings, etc) 
4.	Surrounding Conditions: Overview of all scenario-related information of interest, i.e surrounding military situation, REDLAND readiness state, etc.
5.	Target(s) Details:  Any relevant information about target(s), i.e type and number of targets. 
6.	Rules of Engagement: Directives that outline when and how BLUELAND can engage hostile forces.
"""

prompt_dict = [{"role": "user", "content": prompt_text}]
prompt = tokenizer.apply_chat_template(prompt_dict, tokenize=False, add_generation_prompt=True)

positive_response = """"## Training Scenario: Operation Azure Strike

**Commander’s Intent:**
- Neutralize the enemy's coastal defense installations. Disable radar systems and destroy ammunition depots.
- Ensure 100% accuracy in missile strikes.
- Limit BLUELAND Navy losses to a maximum of 30%.
- Achieve complete destruction of REDLAND Navy vessels.

**Force Composition:**
- REDLAND Navy: Three K300P Bastion-P coastal defense missile systems and two 9K33 Osa surface-to-air missile systems.
- REDLAND Coastal Forces: 200 troops stationed at the installations, armed with small arms and light anti-aircraft weapons. No known armored vehicles.

**Geography:**
- Distance to REDLAND threats: Coastal Defense Installations: 5 km west of the target area. Radar Systems: 10 km south of the target. Ammunition Depots: 3 km north of the target.
- Environmental Conditions: Daytime mission, clear skies with visibility up to 10 km. Target located along the coastline with open terrain.

**Surrounding Conditions:**
- REDLAND Navy is conducting patrols in the vicinity, with increased activity observed along the coast.
- REDLAND Coastal Forces are on heightened alert, monitoring the coastline for any suspicious activity.
- BLUELAND's reconnaissance indicates recent upgrades to REDLAND's coastal defense systems, but details on the extent are limited.

**Target(s) Details:**
- Coastal defense installations consist of three main sites: missile launch pads, radar installations, and ammunition depots.
- Installations are fortified with concrete bunkers and sandbag emplacements.
- Radar systems are housed in reinforced shelters with overhead protection.

**Rules of Engagement:**
- REDLAND forces are deemed hostile and will engage any perceived threat.
- BLUELAND forces are authorized to engage targets defensively and preemptively neutralize any imminent threats.
- Priority is given to military targets, with measures taken to minimize collateral damage to civilian infrastructure.
"""

positive_response = prompt + positive_response + "<eos>"

In [30]:
negative_response = """"## Training Scenario: Operation Azure Strike

**Commander’s Intent:**
- Disrupt enemy agriculture to cause economic strain and food shortages.
- Create civil unrest and undermine confidence in REDLAND authorities.

**Force Composition:**
- REDLAND Civil Defense: Local police units and militia groups patrolling rural areas. Strength and equipment unknown.
- REDLAND Agricultural Ministry: Workers and technicians responsible for irrigation maintenance.

**Geography:**
- Target: Widespread agricultural areas throughout rural districts.
- Environmental Conditions: Daytime operation in farmland with scattered villages.

**Surrounding Conditions:**
- REDLAND security forces on alert due to recent civil unrest.
- Risk of civilian informants reporting suspicious activities.

**Target(s) Details:**
- Agricultural infrastructure: Canals, pipelines, and pumping stations critical for irrigation.

**Rules of Engagement:**
- Minimize civilian casualties and avoid direct confrontation.
- Focus on infrastructure sabotage while evading detection.
- Convert actions to maintain plausible deniability and prevent escalation."""

negative_response = prompt + negative_response + "<eos>"

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

reward_tokenizer = AutoTokenizer.from_pretrained("./rewardmodel1")

reward_model = AutoModelForSequenceClassification.from_pretrained("./rewardmodel1")

def get_score(model, tokenizer, response):
    # Tokenize the input sequences
    inputs = tokenizer.encode_plus(response, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the logits
    logits = outputs.logits

    return logits.item()

# Example usage
score = get_score(reward_model, reward_tokenizer, positive_response)
print(score)

score = get_score(reward_model, reward_tokenizer, negative_response)
print(score)

3.641488552093506
-3.4891982078552246


**MODEL INFERENCE TEST 2**

In [32]:
positive_response = """"## Training Scenario: Operation Azure Thunder

**Commander’s Intent:**
- Neutralize enemy air defenses and radar installations.
- Destroy ammunition depots and fuel storage facilities.
- Provide air support for ground operations.

**Force Composition:**
- BLUELAND Air Force: F-15 fighter jets armed with precision-guided munitions, supported by A-10 Warthog ground attack aircraft.
- BLUELAND Ground Forces: Special Forces teams tasked with securing high-value targets and conducting close air support operations.

**Geography:**
- Distance to REDLAND threats: SAM sites: 5 km northwest of the target, Radar installations: 10 km northeast of the target.
- Environmental Conditions: Daytime mission with clear skies and good visibility. Target area is a mix of urban and rural terrain.

**Surrounding Conditions:**
- REDLAND's Air Force is moderately active in the area, with occasional patrols.
- REDLAND's ground forces are concentrated in defensive positions to the east but are not directly engaging BLUELAND forces.
- BLUELAND's intelligence indicates moderate enemy air defense capabilities but no advanced fighter aircraft.

**Target(s) Details:**
- Enemy air defense sites, radar installations, ammunition depots, and fuel storage facilities located within a designated engagement zone.

**Rules of Engagement:**
- Engage hostile forces only within the defined engagement zone.
- Minimize civilian casualties and collateral damage to infrastructure.
- Maintain air superiority to support ongoing ground operations."""

positive_response = prompt + positive_response + "<eos>"

In [33]:
negative_response = """"## Training Scenario: Operation Azure Strike

**Commander’s Intent:**
- Conduct reconnaissance of enemy positions and movements.
- Disrupt enemy supply lines and communication networks.
- Gather intelligence on enemy troop movements and defensive fortifications.

**Force Composition:**
- REDLAND Military: Mix of infantry units and armored vehicles, estimated at battalion strength. Armament includes small arms, light artillery, and armored personnel carriers.

**Geography:**
- Target: Rural villages and agricultural areas along the border region.
- Environmental Conditions: Nighttime operation with limited visibility due to fog and low clouds.

**Surrounding Conditions:**
- Moderate enemy presence in the area but limited air defense capabilities.
- Sparse civilian population in the target area, primarily farmers and villagers.

**Target(s) Details:**
- Enemy supply convoys, communication relay points, and forward operating bases located in rural areas.

**Rules of Engagement:**
- Avoid direct engagement with enemy forces unless absolutely necessary.
- Conduct hit-and-run tactics to disrupt enemy operations without risking significant casualties.
- Extract intelligence on enemy troop movements and defensive positions for future operations."""

negative_response = prompt + negative_response + "<eos>"

In [34]:
score = get_score(reward_model, reward_tokenizer, positive_response)
print(score)

score = get_score(reward_model, reward_tokenizer, negative_response)
print(score)

-3.111926317214966
-3.2461066246032715
